In [1]:
#This notebook builds a ensemble from baseline randomforest using 
#Morgan fingerprint 
#and cell painting

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008


In [4]:
list_of_lists_df["simple_pred"]=round((list_of_lists_df["fp_proba_scaled"] + list_of_lists_df["CP_proba_scaled"])/2)
list_of_lists_df["Ensemble_Correct"] = list_of_lists_df["simple_pred"]==list_of_lists_df["true"]
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659,0.0,False
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150,0.0,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009,0.0,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224,0.0,True
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325,1.0,False
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705,0.0,False
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008,0.0,True


In [5]:
list_of_lists_df.to_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv", index=False)